In [1]:
import requests
import os
from os import mkdir
from os.path import isdir
import csv
from datetime import datetime
from datetime import timedelta
import time
import sys
import re
class Crawler():
    def __init__(self,dir='data/chips'):
        # save in directory 'data'
        if not isdir(dir):
            mkdir(dir)
        self.dir = dir
    
    def writeCSV(self,file,rows):
        with open("{}/{}.csv".format(self.dir,file),'w',encoding='utf-8') as f:
            writer = csv.writer(f,lineterminator='\n')
            writer.writerows(rows)

    def getChipByDate(self,date,file):
        twse = "https://www.twse.com.tw/fund/T86"
        para = {
            'response' : 'csv',
            'date' : date,
            'selectType' : 'ALL'
        }
        try:
            result = requests.get(twse,params=para)
        except BaseException:
            return date+'Connection fail!'
        if not result.ok:
            return date+'Connection fail!'
        if len(result.text) < 10:
            return date+' No DATA!'
        rows = result.text.replace('\r','').split('\n')
        r_rows = [['#==#'+date]]
        for i in range(len(rows)):
            #row cleanup and split
            row = re.sub('[,"=]','',rows[i].replace(',"','_')).replace('_',',').split(',')
            if (re.search(r'[^A-Za-z0-9]+',row[0]) == None) and (len(row) != 1):
                r_rows.append(row)
        self.writeCSV(file,r_rows)
        return None

In [ ]:
from datetime import datetime
from datetime import timedelta
import time
import csv
oneday = timedelta(days=1)
date = datetime(2019,10,9)
final = datetime(2012,5,2)
file = str(date)[:10].replace('-','')

while(date>final):
    c = Crawler()
    ds = str(date)[:10].replace('-','')
    s = c.getChipByDate(ds,'days/{}'.format(ds))
    if s != None:
        print('{} - Skipped'.format(str(date)[:10]))
    else:
        print('{} - complete'.format(str(date)[:10]))
    time.sleep(1.2)
    date -= oneday

In [ ]:

startIndex =0
endIndex = 1000000
total_index = 0
s_dates = ''
with open('data/chips/20191009.csv','rb') as fin:
    while(1):
        fin.seek(startIndex,0)
        #print(fin.tell())
        data = fin.read(endIndex)
        rows=''
        try:
            rows = data.decode('utf-8').split('#==#')[1]
        except UnicodeDecodeError as err:
            endIndex-=1
            continue
        except IndexError as ierr:
            break
        startIndex += len(rows.encode('utf-8'))+4
        if len(rows) <= 10:
            continue
        ss = rows.split('\n')
        if len(ss)>2:
            s_dates += ss[0]
print(s_dates)


In [ ]:
#crawl from today
oneday = timedelta(days=1)
date = datetime.today()
final = datetime(2012,5,2)
file = str(date)[:10].replace('-','')
while(date>final):
    if str(date)[:10].replace('-','') in s_dates:
        print(str(date)+'  exist')
        date -= oneday 
        continue
    c = Crawler()
    ds = str(date)[:10].replace('-','')
    input()
    s = c.getChipByDate(ds,'days/{}'.format(file))
    print('{} - complete'.format(str(date)[:10]))
    time.sleep(1.2)
    date -= oneday    

In [5]:
#check and crawl from indicators
import pandas as pd
import os
def minguo2Date(ming):
    y , m , d = ming.split('/')
    return str(int(y)+1911)+m+d
exdays = ''
for f in os.listdir('data/chips/days'):
    exdays += f[:-4]
oneday = timedelta(days=1)
date = datetime.today()
final = datetime(2012,5,2)
file = str(date)[:10].replace('-','')
df = pd.read_csv('data/indicators/2317_indicators.csv')
dist_days = df.iloc[:,0]
index=0
for i,d in enumerate(dist_days.values):
    if minguo2Date(d) == '20120502':
        index = i
dist_days = dist_days.iloc[index:].values
for d in dist_days:
    d = minguo2Date(d)
    if d in exdays:
        continue
    c = Crawler()
    s = c.getChipByDate(d,'days/{}'.format(d))
    if s != None:
        print('{} - Skipped'.format(d))
    else:
        print('{} - complete'.format(d))
    time.sleep(1.2)

In [ ]:
#split data from daily file by stock number
stock_no = '23172330'
import os
import csv
import re
path = 'data/chips/days/'
flag = True
for f in os.listdir(path):
    rows = []
    with open(path+f,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        for r in reader:
            rows.append(str(r))
        date = rows[0][6:-2]
        for row in rows:
            cols = [date]
            r = re.sub("[ \[\]\r']",'',row).split(',')
            if re.search(r'[^A-Za-z0-9]+',r[0]) != None:
                continue
            #write only needed
            if not (r[0] in stock_no):
                continue
            stockno = r[0]
            cols.extend(r[2:])
            with open('data/chips/chips_{}.csv'.format(stockno),'a') as f:
                writer = csv.writer(f,lineterminator='\n')
                writer.writerow(cols)
        print(date+' -- Written!')